In [12]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [13]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [14]:
df = pd.merge(movies,ratings, how="left",on="movieId")
df=df.fillna(0)
df.head(10)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0,5.0,8.590469e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,1.303501e+09
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,5.0,8.586109e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.0,8.508158e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.0,8.517663e+08
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,5.0,1.350207e+09
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,28.0,3.0,8.840984e+08
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,29.0,4.0,8.469426e+08
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,30.0,4.5,1.292690e+09
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,31.0,4.0,8.323975e+08


In [15]:
df=df.drop(["timestamp"],axis=1)

In [16]:
df

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0,5.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.0
...,...,...,...,...,...
105338,148238,A Very Murray Christmas (2015),Comedy,475.0,3.0
105339,148626,The Big Short (2015),Drama,458.0,4.0
105340,148626,The Big Short (2015),Drama,576.0,4.5
105341,148626,The Big Short (2015),Drama,668.0,4.5


In [17]:
train, test = train_test_split(df,test_size=0.2,random_state=0)

In [40]:
class popularity():
    def __init__(self, train, test, userId, movieId):
        self.train = train
        self.test = test
        self.movieId = movieId
        self.userId = userId
        self.pop_recommendation = None
        
    def fit(self):
        train_group = train.groupby([self.movieId]).agg(self.userId).count().reset_index()
        train_group.rename(columns = {"userId":"score"}, inplace = True)
        train_sorted = train_group.sort_values(["score",self.movieId],ascending = [0,1])
        train_sorted["Rank"]=train_sorted["score"].rank(ascending=0,method="first")
        self.pop_recommendation = train_sorted
        
    def recommend(self, userId, n = 10):
        user_recommend = self.pop_recommendation
        movies_rated = self.train.loc[self.train[self.userId]==userId][self.movieId]
        user_recommend = user_recommend[~user_recommend[self.movieId].isin(movies_rated)]
        user_recommend["userId"] = userId
        co = user_recommend.columns.tolist()
        user_recommend = user_recommend[co]
        return user_recommend.head(n)

In [41]:
p_recomm = popularity(train=train, test=test, userId="userId", movieId="movieId")

In [42]:
p_recomm.fit()
test_1 = p_recomm.recommend(628)

In [43]:
test_1

,movieId,score,Rank,userId
274,318,247,2.0,628
416,480,236,4.0,628
98,110,213,7.0,628
933,1196,189,12.0,628
2308,2959,165,20.0,628
507,588,162,23.0,628
1004,1270,155,29.0,628
1233,1580,148,32.0,628
333,380,138,35.0,628
823,1036,137,37.0,628
